<a href="https://colab.research.google.com/github/jeffheaton/app_generative_ai/blob/main/t81_559_class_06_4_qa.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# T81-559: Applications of Generative Artificial Intelligence
**Module 6: Retrieval-Augmented Generation (RAG)**
* Instructor: [Jeff Heaton](https://sites.wustl.edu/jeffheaton/), McKelvey School of Engineering, [Washington University in St. Louis](https://engineering.wustl.edu/Programs/Pages/default.aspx)
* For more information visit the [class website](https://sites.wustl.edu/jeffheaton/t81-558/).

# Module 6 Material

* Part 6.1: Introduction to Retrieval-Augmented Generation (RAG) [[Video]]() [[Notebook]](t81_559_class_06_1_rag.ipydb)
* Part 6.2: Introduction to ChromaDB [[Video]]() [[Notebook]](t81_559_class_06_2_chromadb.ipynb)
* Part 6.3: Understanding Embeddings [[Video]]() [[Notebook]](t81_559_class_06_3_embeddings.ipynb)
* **Part 6.4: Question Answering Over Documents** [[Video]]() [[Notebook]](t81_559_class_06_4_qa.ipynb)
* Part 6.5: Embedding Databases [[Video]]() [[Notebook]](t81_559_class_06_5_embed_db.ipynb)


# Google CoLab Instructions

The following code ensures that Google CoLab is running and maps Google Drive if needed.

In [11]:
import os

try:
    from google.colab import drive, userdata
    COLAB = True
    print("Note: using Google CoLab")
except:
    print("Note: not using Google CoLab")
    COLAB = False

# OpenAI Secrets
if COLAB:
    os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')

# Install needed libraries in CoLab
if COLAB:
    !pip install langchain langchain_openai chromadb langchain_community sentence-transformers langchainhub

Note: using Google CoLab


# 6.4: Question Answering Over Documents

Retrieval-Augmented Generation (RAG) is an advanced technique designed to enhance the capabilities of large language models (LLMs) by integrating external data into the response generation process. For RAG to be truly effective, it must operate over data that is not already present in the foundation model. This is crucial because the primary advantage of RAG lies in its ability to pull in specific, often up-to-date information that the LLM might not have been exposed to during its initial training.

This feature makes RAG particularly valuable for corporate environments. Companies generate and store vast amounts of proprietary data, including internal documents, customer information, and detailed reports. This data is often unique to the organization and not part of the publicly available corpus that an LLM would be trained on. By using RAG, businesses can leverage their own data to obtain more precise and contextually relevant answers from their LLMs, thereby improving decision-making processes and operational efficiency.

To illustrate the capabilities of RAG, we will utilize a sample dataset I created, containing synthetic data of employee biographies from five fictional companies. This dataset is crafted to demonstrate how RAG can effectively retrieve and utilize specific information that a foundation model would not inherently possess. By doing so, it provides a clear example of how RAG can be applied in a real-world corporate setting.

Here is an example of such a generated biography:

> Elena Martinez is a seasoned Robotics Engineer at FutureTech, a leading innovator in artificial intelligence and robotics based in Silicon Valley. With a Master's degree in Mechanical Engineering from MIT and over a decade of experience, Elena has been pivotal in the development of autonomous robotic systems designed to enhance urban mobility and accessibility. Her groundbreaking work includes the creation of the first AI-powered robotic assistant that can seamlessly interact with urban environments to aid the elderly and disabled. A passionate advocate for women in STEM, Elena also leads FutureTech's outreach program, aiming to inspire the next generation of female engineers through workshops and mentorships. Her contributions have not only propelled FutureTech to new heights but have also set new standards in robotics applications for social good.


This biography showcases the type of detailed, company-specific information that RAG can retrieve and incorporate into its responses, enhancing the relevance and accuracy of the generated content. Through the integration of such tailored data, RAG not only enriches the output of LLMs but also ensures that the responses are aligned with the unique context and needs of the organization.

This sample data is stored at the following URL's, each file holds people from one of the companies.

* https://data.heatonresearch.com/data/t81-559/bios/DD.txt
* https://data.heatonresearch.com/data/t81-559/bios/FT.txt
* https://data.heatonresearch.com/data/t81-559/bios/GS.txt
* https://data.heatonresearch.com/data/t81-559/bios/NGS.txt
* https://data.heatonresearch.com/data/t81-559/bios/TI.txt

The following code defines these URL's and instanciates an LLM model.



In [12]:
from langchain.chains.summarize import load_summarize_chain
from langchain.document_loaders import TextLoader
from langchain import OpenAI, PromptTemplate
from langchain_openai import ChatOpenAI
from IPython.display import display_markdown
from langchain.indexes import VectorstoreIndexCreator
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores.inmemory import InMemoryVectorStore
from langchain.schema import Document
import requests

MODEL = 'gpt-4o-mini'

llm = ChatOpenAI(
        model=MODEL,
        temperature=0.2,
        n=1
    )

urls = [
    "https://data.heatonresearch.com/data/t81-559/bios/DD.txt",
    "https://data.heatonresearch.com/data/t81-559/bios/FT.txt",
    "https://data.heatonresearch.com/data/t81-559/bios/GS.txt",
    "https://data.heatonresearch.com/data/t81-559/bios/NGS.txt",
    "https://data.heatonresearch.com/data/t81-559/bios/TI.txt"
]


The function processes a large document by dividing it into smaller, manageable segments, known as chunks, to create embeddings for LLM retrieval in Retrieval-Augmented Generation (RAG). The chunk parameter determines the maximum length of each segment, ensuring the text is broken down into parts that can be efficiently handled and analyzed by the language model. The overlap parameter specifies the number of tokens that should be repeated at the beginning of each new chunk, creating an overlap between consecutive chunks. This overlap helps maintain contextual continuity across chunks, improving the quality and accuracy of the embeddings. By systematically segmenting the document and generating embeddings for each chunk, the function enhances the language model's ability to retrieve relevant information, leading to more precise and contextually appropriate responses in the RAG framework.

In [13]:
def chunk_text(text, chunk_size, overlap):
    chunks = []
    for i in range(0, len(text), chunk_size - overlap):
        chunks.append(text[i:i + chunk_size])
    return chunks

In this function, we set the parameters for chunk size and overlap to the following values:

In [14]:
chunk_size = 900
overlap = 300

We are processing text files that contain lists of people's biographies. These settings help manage the large text data more effectively. The chunk size of 1000 ensures that each segment of text, or chunk, contains up to 1000 tokens, making it easier for the language model to handle and generate embeddings for each part. The overlap of 200 tokens means that each new chunk will start with the last 200 tokens from the previous chunk. This overlap is crucial in maintaining the continuity of the context between chunks, which is particularly useful when dealing with biographical data where details often span across multiple chunks.


The following code reads text content from a list of URLs, processes this content by splitting it into smaller, manageable chunks, and stores these chunks as Document objects. Initially, an empty list named documents is created to hold these Document objects. The code then iterates over each URL in the provided list, printing a message to indicate the current URL being processed. For each URL, it fetches the content using the requests.get method and checks for any HTTP errors with response.raise_for_status(). Once the content is successfully retrieved, it is split into smaller segments using the chunk_text function, which takes into account the specified chunk size and overlap to maintain contextual continuity between chunks. These chunks are then used to create Document objects, each containing a portion of the text. These Document objects are subsequently appended to the documents list, effectively organizing the large text files into smaller, retrievable segments suitable for further processing, such as generating embeddings for LLM retrieval in Retrieval-Augmented Generation (RAG).

In [15]:
documents = []

for url in urls:
    print(f"Reading: {url}")
    response = requests.get(url)
    response.raise_for_status()  # Ensure we notice bad responses
    content = response.text
    chunks = chunk_text(content, chunk_size, overlap)
    for chunk in chunks:
        document = Document(page_content=chunk)
        documents.append(document)

Reading: https://data.heatonresearch.com/data/t81-559/bios/DD.txt
Reading: https://data.heatonresearch.com/data/t81-559/bios/FT.txt
Reading: https://data.heatonresearch.com/data/t81-559/bios/GS.txt
Reading: https://data.heatonresearch.com/data/t81-559/bios/NGS.txt
Reading: https://data.heatonresearch.com/data/t81-559/bios/TI.txt


We've loaded the biography data and are now integrating it into the ChromaDB database using the following code. ChromaDB, requiring an embeddings model, defaults to "all-MiniLM-L6-v2". Other [model wrappers](https://docs.trychroma.com/integrations/openai) are provided. This model is provided by SentenceTransformerEmbeddings, which is instantiated here for embedding functions. Utilizing CharacterTextSplitter from langchain_text_splitters, documents are chunked into manageable parts for processing. Chroma, a vector store, interfaces seamlessly with these components, enabling efficient storage and retrieval of embeddings within the database.

In [ ]:
from langchain_text_splitters import CharacterTextSplitter
from langchain_community.embeddings.sentence_transformer import (
    SentenceTransformerEmbeddings,
)
from langchain.vectorstores import Chroma

text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
docs = text_splitter.split_documents(documents)

embedding_function = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")

db = Chroma.from_documents(docs, embedding_function)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


We're set to construct a RAG (Retrieval-Augmented Generation) chain with the following code. Starting with a retriever initialized from our previously configured ChromaDB database, documents are formatted into a concatenated string for input preparation. The chain incorporates a question-answer flow: the formatted documents serve as context, alongside a pass-through question handler. Utilizing the RAG model retrieved from the hub, the chain proceeds to a language model (llm). Finally, results are parsed into a string format using StrOutputParser, completing the process for generating answers based on retrieved document contexts.

In [ ]:
from langchain import hub
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

rag_prompt = hub.pull("rlm/rag-prompt")

def format_documents(documents):
    return "\n\n".join(doc.page_content for doc in documents)

retriever = db.as_retriever()

qa_chain = (
    {"context": retriever | format_documents, "question": RunnablePassthrough()}
    | rag_prompt
    | llm
    | StrOutputParser()
)

We can now invoke the RAG chain and query it about one of the people who were inside of our sampe biography data.

In [ ]:
qa_chain.invoke("What company does Elena Martinez work for?")

## Taking Apart the RAG Chain

Several components combine to bring external data into a prompt for RAG-style access. In this section, we will examine each and see how it functions individually. We begin by seeing how to prompt ChromaDB and watch it retrieve relevant information from the larger set of documents. This smaller subset allows the data to fit into the context. Even if the entirety of the source material would fit into the context buffer of the LLM, RAG would decrease costs and improve access time by requiring much less data to be sent to the LLM.

In [ ]:
# query it
query = "What company does Elena Martinez work for?"
docs = db.similarity_search(query)

# print results
print(docs)

As you can see, ChromaDB returns multiple bits of information that will give the LLM context to answer the question. None of this information is "common knowldge," it only exists in the generated biography data we loaded into ChromaDB.

We will also look at the RAG prompt template. We use the standard prompt template provided by LangChain Hub. The text of this prompt is here. Both the question and any supporting information from the database are provided.

In [30]:
rag_prompt = hub.pull("rlm/rag-prompt")
rag_prompt.messages[0].prompt.template

"You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.\nQuestion: {question} \nContext: {context} \nAnswer:"

The R in RAG standards stands for "retrieval." Let's pass the question to the retriever object, which will query ChromaDB to find which documents most closely match. You will see some overlap and duplication. This overlap helps to ensure continuity across chunk boundaries.

In [31]:
retriever.invoke("What company does Elena Martinez work for?")

[Document(page_content="Elena Martinez is a seasoned Robotics Engineer at FutureTech, a leading innovator in artificial intelligence and robotics based in Silicon Valley. With a Master's degree in Mechanical Engineering from MIT and over a decade of experience, Elena has been pivotal in the development of autonomous robotic systems designed to enhance urban mobility and accessibility. Her groundbreaking work includes the creation of the first AI-powered robotic assistant that can seamlessly interact with urban environments to aid the elderly and disabled. A passionate advocate for women in STEM, Elena also leads FutureTech's outreach program, aiming to inspire the next generation of female engineers through workshops and mentorships. Her contributions have not only propelled FutureTech to new heights but have also set new standards in robotics applications for social good.\n\nJason Carter is a Senior Robotics Engineer at FutureTech, a leading company in innovative technology solutions 

This data is combined with the question into the RAG prompt to be submitted to the LLM.